In [ ]:
!pip install --upgrade --quiet langchain langchain-community langchain-groq neo4j


In [ ]:
NEO4J_URI = "neo4j://127.0.0.1:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "gihanlakmal"

In [ ]:
import os
os.environ["NEO4J_URI"]=NEO4J_URI
os.environ["NEO4J_USERNAME"]=NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"]=NEO4J_PASSWORD

In [ ]:
from langchain_community.graphs import Neo4jGraph


In [ ]:
graph=Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [ ]:
from langchain.document_loaders import WikipediaLoader
raw_document = WikipediaLoader(query="Elizabath queen").load()

In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    api_key="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",
    model="deepseek-r1-distill-llama-70b"
)



In [165]:
total = 0
for i in range(6):
    total += len(raw_document[i].page_content)
print(f"Total number of characters in the first 6 documents: {total}")

Total number of characters in the first 6 documents: 15528


In [ ]:
from langchain.text_splitter import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(raw_document[:3])


In [ ]:
documents[1].page_content

In [ ]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [ ]:
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [ ]:
graph_documents

In [ ]:
graph.add_graph_documents(graph_documents , baseEntityLabel=True , include_source=True)

In [ ]:
default_query = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s , r , t LIMIT 50"


In [ ]:
from neo4j import GraphDatabase
from pyvis.network import Network
from IPython.display import display, HTML

def show_graph(cypher: str):
    driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USERNAME, NEO4J_PASSWORD))
    session = driver.session()

    result = session.run(cypher)

    net = Network(notebook=True, height="600px", width="100%", bgcolor="#222222", font_color="white")

    nodes = set()
    edges = []

    for record in result:
        for key in record.keys():
            value = record[key]
            # Handle Nodes
            if hasattr(value, 'element_id') and hasattr(value, 'labels'):
                node_id = str(value.element_id)
                if node_id not in nodes:
                    label = value.get("name") or value.get("title") or f"Node {node_id}"
                    net.add_node(node_id, label=label)
                    nodes.add(node_id)
            # Handle Relationships
            elif hasattr(value, 'start_node') and hasattr(value, 'end_node'):
                start_id = str(value.start_node.element_id)
                end_id = str(value.end_node.element_id)
                rel_type = value.type
                edges.append((start_id, end_id, rel_type))

    # Add edges
    for start_id, end_id, rel_type in edges:
        if start_id in nodes and end_id in nodes:
            net.add_edge(start_id, end_id, label=rel_type)

    session.close()

    net.show("graph.html")
    display(HTML("graph.html"))



In [ ]:
show_graph(default_query) 

In [ ]:
display(HTML("graph.html"))

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)



In [ ]:
from typing import Tuple , List , Optional

In [ ]:
from langchain_community.vectorstores import Neo4jVector

In [ ]:
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
vector_index = Neo4jVector.from_existing_graph(
    embedding=embedding,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)



In [ ]:
from pydantic import BaseModel, Field


# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities."""
    
    names: List[str] = Field(
            ...,
            description="All the person, organization, or business entities that "
            "appear in the text",
            )

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate

In [ ]:
prompt  = ChatPromptTemplate.from_messages(
    [
        
        ("system", "You are extracting organization and person entities from the text."),
        ("human", "Use the given format to extract information from the following "  "input: {question}" ),
        
    ]
    
)

In [ ]:
entity_chain = prompt | llm.with_structured_output( Entities)

In [ ]:
entity_chain.invoke({"question": "Who is the Elizabath?"})

In [ ]:
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

In [ ]:
def generate_full_text_query(input: str) -> str:
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f"{word}~2 AND"
    full_text_query += f"{words[-1]}~2"
    return full_text_query.strip()
    

In [142]:
def structured_retriever(question: str) -> list[str]:
    # Step 1: Extract entities
    entities = entity_chain.invoke({"question": question})

    # Step 2: Initialize result string
    results = ""

    # Step 3: Query Neo4j for each entity
    for entity in entities.names:
        response = graph.query(
            """
            CALL db.index.fulltext.queryNodes('keyword', $query, {limit: 2})
            YIELD node, score
            CALL {
                WITH node
                MATCH (node)-[r:MENTIONS]->(neighbor)
                RETURN node.id + ' -[' + type(r) + ']-> ' + neighbor.id AS output
                UNION
                MATCH (node)<-[r:MENTIONS]-(neighbor)
                RETURN neighbor.id + ' -[' + type(r) + ']-> ' + node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        results += "\n".join([el["output"] for el in response])

    return results

In [143]:
print(structured_retriever("Who is the Elizabath ?"))

63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Queen Elizabeth Hospital
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> King'S Lynn
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Norfolk
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> England
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> West Norfolk
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> South Lincolnshire
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Fenland District
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Cambridgeshire
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Queen Elizabeth Hospital King'S Lynn Nhs Foundation Trust
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Queen Elizabeth The Queen Mother
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Reinforced Autoclaved Aerated Concrete Plank Roof
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Fermoy Unit
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Sandringham
63edb6a89cb9d7a07b7f9db5d26ee613 -[MENTIONS]-> Arthur Levin Day Surgery Centre
63edb6a89cb9d7a07b7f9db5d26e

In [144]:
def retriever(question: str):
    print(f"search query: {question}")
    structured_data= structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"Document ".join(unstructured_data)}
"""
    return final_data

In [145]:

_template = """Given the following conversation and a follow up question , rephrase the follow up question to be a standalone question,
in its original langugae .
Chat History:
{chat_history}Follow Up Input: {question}
Rephrased Question:"""

In [146]:
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

from langchain_core.messages import HumanMessage, AIMessage

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

In [149]:
from langchain_core.runnables import RunnableBranch, RunnableLambda, RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

In [151]:


# Assume this function is defined somewhere
def _format_chat_history(history):
    return "\n".join([f"{msg['role']}: {msg['content']}" for msg in history])

# Example prompt (should be a valid PromptTemplate)
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template) # Your PromptTemplate here

_search_query = RunnableBranch(
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | llm 
        | StrOutputParser(),
    ),
    RunnableLambda(lambda x: x["question"]),
)

In [161]:
template = """
You are an intelligent assistant helping users answer factual questions.

Context:|
{context}

Answer the following question using only the information from the context above.

Question: {question}

Write the answer in a **clear, well-structured paragraph**, without listing steps or saying "let me think." Avoid verbose or repeated parts. Keep it informative and natural.

Answer:"""



prompt = ChatPromptTemplate.from_template(template)

In [162]:
from langchain_core.runnables import RunnableParallel, RunnableLambda

chain = (
    RunnableParallel(
        {
            "context": _search_query |retriever , 
            "question": RunnablePassthrough(),
        }
    )
    
    |prompt
    |llm
    |StrOutputParser()
)

In [164]:
chain.invoke({
    "question": "who is the Elizabath  ? "})

search query: who is the Elizabath  ? 


'<think>\nAlright, I need to figure out who "Elizabeth" refers to based on the provided context. Let me go through the information step by step.\n\nFirst, looking at the structured data, there are mentions of "Queen Elizabeth Hospital" and "Queen Elizabeth The Queen Mother." This suggests that there are two prominent Elizabeths: Queen Elizabeth II and her mother, also named Elizabeth.\n\nIn the unstructured text, it\'s mentioned that the hospital is named after Queen Elizabeth The Queen Mother, not Queen Elizabeth II. This clarifies that there are two different Elizabeths involved. Additionally, the text talks about events involving Queen Elizabeth II, such as her visiting the hospital and opening certain units, which indicates she\'s a separate individual.\n\nThere\'s also mention of Princess Elizabeth, who later became Queen Elizabeth II, in the context of the cottage Y Bwthyn Bach, which was a gift to her when she was a princess. This further distinguishes between the two Elizabeths